In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab import data_table
data_table.enable_dataframe_formatter()

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# 데이터셋 로드
df =  pd.read_csv('/content/drive/MyDrive/cp2_log_data.csv', parse_dates=['event_time'])

In [ ]:
# 칼럼 평점화 진행
df_v = df.copy()

# 필요한 컬럼만 사용

df_v = df_v[['user_id','category_code','event_type']]
df_v

,user_id,category_code,event_type
0,541312140,unknown.unknown.unknown,view
1,554748717,appliances.environment.water_heater,view
2,519107250,furniture.living_room.sofa,view
3,550050854,computers.notebook,view
4,535871217,electronics.smartphone,view
...,...,...,...
38672585,537931532,electronics.camera.video,view
38672586,527322328,unknown.unknown.unknown,view
38672587,566280422,auto.accessories.player,view
38672588,513118352,electronics.clocks,view


In [ ]:
df_v = df_v.drop_duplicates()
df_v.event_type.value_counts()

view        5794964
purchase     403409
cart         376926
Name: event_type, dtype: int64

In [ ]:
event_type_strength = {
   'view': 1.5,
   'cart': 1.6, 
   'purchase': 1.9
}

df_v['event_type'] = df_v['event_type'].apply(lambda x: event_type_strength[x])

# 참고
# https://www.kaggle.com/code/gspmoreira/recommender-systems-in-python-101/notebook.

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [ ]:
# 선호도 확인을 위한 그룹화 진행

df_v_g = df_v.groupby(['user_id','category_code'])['event_type'].sum().reset_index()
df_v_g

,user_id,category_code,event_type
0,33869381,kids.carriage,1.5
1,184265397,furniture.living_room.chair,1.5
2,184265397,unknown.unknown.unknown,1.5
3,195082191,electronics.audio.headphone,1.5
4,200673532,computers.components.motherboard,1.5
...,...,...,...
5795272,566280536,furniture.living_room.sofa,1.5
5795273,566280663,electronics.smartphone,1.5
5795274,566280676,furniture.bedroom.bed,1.5
5795275,566280697,electronics.camera.video,1.5


In [ ]:
# 초기 데이터셋에 평점 붙이기

df_v_m = pd.merge(left=df, right=df_v_g,how='left',\
                    on=['user_id','category_code'],sort=False)

df_v_m

,Unnamed: 0,event_time,event_type_x,product_id,category_id,category_code,brand,price,user_id,user_session,division1,division2,division3,event_type_y
0,0,2019-10-01 00:00:00+00:00,view,44600062,2103807459595387724,unknown.unknown.unknown,shiseido,35.79,541312140,72d76fde-8bb3-4e00-8c23-a032dfed738c,unknown,unknown,unknown,3.4
1,1,2019-10-01 00:00:00+00:00,view,3900821,2053013552326770905,appliances.environment.water_heater,aqua,33.20,554748717,9333dfbd-b87a-4708-9857-6336556b0fcc,appliances,environment,water_heater,1.5
2,2,2019-10-01 00:00:01+00:00,view,17200506,2053013559792632471,furniture.living_room.sofa,unknown,543.10,519107250,566511c2-e2e3-422b-b695-cf8e6e792ca8,furniture,living_room,sofa,1.5
3,3,2019-10-01 00:00:01+00:00,view,1307067,2053013558920217191,computers.notebook,lenovo,251.74,550050854,7c90fc70-0e80-4590-96f3-13c02c18c713,computers,notebook,notebook,1.5
4,4,2019-10-01 00:00:04+00:00,view,1004237,2053013555631882655,electronics.smartphone,apple,1081.98,535871217,c6bd7419-2748-4c56-95b4-8cec9ff8b80d,electronics,smartphone,smartphone,1.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38672585,42448759,2019-10-31 23:59:58+00:00,view,2300275,2053013560530830019,electronics.camera.video,gopro,527.40,537931532,22c57267-da98-4f28-9a9c-18bb5b385193,electronics,camera,video,1.5
38672586,42448760,2019-10-31 23:59:58+00:00,view,10800172,2053013554994348409,unknown.unknown.unknown,redmond,61.75,527322328,5054190a-46cb-4211-a8f1-16fc1a060ed8,unknown,unknown,unknown,1.5
38672587,42448761,2019-10-31 23:59:58+00:00,view,5701038,2053013553970938175,auto.accessories.player,kenwood,128.70,566280422,05b6c62b-992f-4e8e-91f7-961bcb4719cd,auto,accessories,player,1.5
38672588,42448762,2019-10-31 23:59:59+00:00,view,21407424,2053013561579406073,electronics.clocks,tissot,689.85,513118352,4c14bf2a-2820-4504-929d-046356a5a204,electronics,clocks,clocks,1.5


In [ ]:
# 중분류가 해당되는 것만 잘라서 사용

df_cat = df_v_m.loc[(df_v_m['division2']=='smartphone')|
                    (df_v_m['division2']=='video')|
                    (df_v_m['division2']=='kitchen')|
                    (df_v_m['division2']=='notebook')
                    ]

In [ ]:
df_cat=df_cat.reset_index(drop=True)
df_cat

,Unnamed: 0,event_time,event_type_x,product_id,category_id,category_code,brand,price,user_id,user_session,division1,division2,division3,event_type_y
0,3,2019-10-01 00:00:01+00:00,view,1307067,2053013558920217191,computers.notebook,lenovo,251.74,550050854,7c90fc70-0e80-4590-96f3-13c02c18c713,computers,notebook,notebook,1.5
1,4,2019-10-01 00:00:04+00:00,view,1004237,2053013555631882655,electronics.smartphone,apple,1081.98,535871217,c6bd7419-2748-4c56-95b4-8cec9ff8b80d,electronics,smartphone,smartphone,1.5
2,9,2019-10-01 00:00:11+00:00,view,1004545,2053013555631882655,electronics.smartphone,huawei,566.01,537918940,406c46ed-90a4-4787-a43b-59a410c1a5fb,electronics,smartphone,smartphone,1.5
3,10,2019-10-01 00:00:11+00:00,view,2900536,2053013554776244595,appliances.kitchen.microwave,elenberg,51.46,555158050,b5bdd0b3-4ca2-4c55-939e-9ce44bb50abd,appliances,kitchen,microwave,1.5
4,11,2019-10-01 00:00:11+00:00,view,1005011,2053013555631882655,electronics.smartphone,samsung,900.64,530282093,50a293fb-5940-41b2-baf3-17af0e812101,electronics,smartphone,smartphone,1.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17292888,42448746,2019-10-31 23:59:54+00:00,view,1002786,2053013555631882655,electronics.smartphone,unknown,391.26,512789086,cc782b99-88ab-4573-8311-c62e1d447757,electronics,smartphone,smartphone,1.5
17292889,42448751,2019-10-31 23:59:56+00:00,view,1005008,2053013555631882655,electronics.smartphone,xiaomi,94.46,557202408,2169c430-9acc-4db8-8061-ac70ff0c66ea,electronics,smartphone,smartphone,1.5
17292890,42448752,2019-10-31 23:59:56+00:00,view,1003936,2053013555631882655,electronics.smartphone,xiaomi,437.33,520088904,4d3b30da-a5e4-49df-b1a8-ba5943f1dd33,electronics,smartphone,smartphone,1.5
17292891,42448757,2019-10-31 23:59:58+00:00,view,1004870,2053013555631882655,electronics.smartphone,samsung,275.25,518956209,6764041a-9285-4869-8a32-a79adf31d212,electronics,smartphone,smartphone,1.5


In [ ]:
df_cat = df_cat.rename(columns={'event_type_y': 'rating'})

In [ ]:
df_cat = df_cat.drop(['Unnamed: 0','event_time','event_type_x','category_code','user_session','division3'], axis=1)

In [ ]:
df_cat.to_csv('last_csv')